# 第4章 频域滤波

- 正弦信号可由幅值、频率和初相位三个参数完全确定。傅里叶变换将信号分解为一系列正弦信号的线性组合，从而得到信号的频谱。时间频率和空间频率是现实世界中两个最重要也是最基本的物理量，如人的嗓门粗细、图像纹理的单调与丰富、图像的清晰度等都与频率有关。傅里叶变换将时间/空间信号与频率联系起来，可以把信号由时域/空域变换到频域、再由频域变换回时域/空域。这样做的目的，一是便于理解信号频率组成，二是便于提取信号特征。
- 离散傅里叶变换DFT(Discrete Fourier Transform)及其快速傅里叶变换FFT算法（Fast Fourier Transform）已成为信号分析和处理的重要工具，也是频域滤波（Frequency Domain Filtering）的理论和技术基础。

## 4.1 离散傅里叶变换DFT

In [2]:
#导入本章示例所用到的包,使用本文档中示例,先运行一次本段代码
#-*- coding: utf-8 -*-
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['STSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

from scipy.fft import fft,ifft,fft2,ifft2,fftshift,ifftshift
from skimage import io,util, exposure
%matplotlib inline

### 4.1.5 离散傅里叶变换的实现
### 示例：一维含噪信号的频谱分析
- 从幅度谱中可以看出，尽管存在噪声，在50赫兹和120赫兹处两个正弦信号的幅值显著，但并非等于0.7和1。
- 信号序列持续时间越长，频谱计算得到的各频率分量幅度越接近原值。

In [ ]:
#采用离散傅里叶变换分析一维含噪时间信号的频率组成

#生成含噪信号序列
Fs = 500   #采样频率Sampling frequency
T = 1/Fs   #采样周期Sampling period
N = 1000   #信号序列长度Length of signal
t = np.linspace(0.0, N*T, N)#采样时刻向量

#生成信号序列
#由幅度为0.7的50Hz正弦信号和幅度为1的120Hz正弦信号相加而成
S = 0.7*np.sin(2*np.pi*50*t)+1* np.sin(2*np.pi*120*t)
#向信号中添加零均值,标准差为2的白噪声
X = S + 2*np.random.standard_normal(t.size)

#计算含噪信号X的DFT变换
Yu = fft(X)
#计算含噪信号X的双边幅度谱,取复数的绝对值，即双边幅度谱
#用因子1/N对幅度谱重新标定
Yu_amp2side = np.abs(Yu/N)

#中心化,得到含噪信号X的N/2中心化双边幅度谱
Yu2 = fftshift(Yu);
Yu_amp2side_centered = np.abs(Yu2/N);

#计算含噪信号X的单边幅度谱
Yu_amp1side = Yu_amp2side [0:int(N/2)].copy()
Yu_amp1side[1:-1] = 2*Yu_amp1side[1:-1]

#显示结果
plt.figure(figsize=(14,10))

#含噪信号X的时域波形图
plt.subplot(2,2,1); plt.plot(1000*t,X,color='black')
#plt.title('Signal Corrupted with Zero-Mean Random Noise')
#plt.xlabel('t (milliseconds)')
#plt.ylabel('X(t)')

plt.xlabel('t (毫秒)')
plt.ylabel('X(t)')

#定义双边幅度谱频率轴的频率范围
f = np.linspace(0.0, Fs, N)             
#含噪信号X的双边幅度谱
plt.subplot(2,2,2); plt.plot(f, Yu_amp2side,color='black')
plt.title('Two-sided Amplitude Spectrum of X(t),Non centered')
plt.xlabel('f (Hz)')
plt.ylabel('|Yu_amp2side(f)|')

#定义0-中心化双边幅度谱频率轴的频率范围
f = np.linspace(0.0, Fs, N)-Fs/2
#含噪信号X的N/2中心化双边幅度谱，以0频点中心化方式显示
plt.subplot(2,2,3); plt.plot(f, Yu_amp2side_centered,color='black')
plt.title('Two-sided Amplitude Spectrum of X(t),0-centered')
plt.xlabel('f (Hz)')
plt.ylabel('|Yu_amp2side_centerd|')

#定义边幅度谱频率轴的频率范围
f = np.linspace(0.0, Fs/2.0, N//2) # 运算符'//'取整除,返回商的整数部分(向下取整)
#含噪信号X的单边幅度谱
plt.subplot(2,2,4); plt.plot(f,Yu_amp1side,color='black')
plt.title('Single-sided Amplitude Spectrum of X(t)')
plt.xlabel('f (Hz)')
plt.ylabel('|Yu_amp1side(f)|')

plt.show()

### 示例：采用SciPy中的fft2函数计算灰度图像的傅里叶频谱

In [ ]:
#采用SciPy中的fft2函数计算灰度图像的傅里叶频谱 

#读入一幅灰度图像
img = cv.imread('.\imagedata\cameraman.tif',cv.IMREAD_GRAYSCALE)

#计算图像的DFT
imgdft = fft2(img)
#计算图像幅度谱
imgdft_mag = np.abs(imgdft)

#对图像频谱中心化处理
imgdft_cent = fftshift(imgdft)
#计算图像中心化幅度谱
imgdft_mag_cent = np.abs(imgdft_cent)

#为便于观察图像幅度谱，采用对数校正增强图像
c1 = 255/np.log10(1+ np.max(imgdft_mag))
imgdft_mag_log = c1 * np.log10(1+ imgdft_mag)
c2 = 255/np.log10(1+ np.max(imgdft_mag_cent))
imgdft_mag_cent_log = c2 * np.log10(1+ imgdft_mag_cent)

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#灰度图像
plt.subplot(1,5,1);plt.imshow(img,vmin=0,vmax=255)
plt.title('Input image')
plt.axis('off')

#未中心化幅度谱
plt.subplot(1,5,2);plt.imshow(imgdft_mag)
plt.title('Magnitude spectrum')
plt.axis('off')

#未增强的中心化幅度谱
plt.subplot(1,5,3);plt.imshow(imgdft_mag_cent)
plt.title('Centered magnitude spectrum')
plt.axis('off')

#对数变换增强的中心化幅度谱
plt.subplot(1,5,4);plt.imshow(imgdft_mag_cent_log,vmin=0,vmax=255)
plt.title('Centered magnitude spectrum,log')
plt.axis('off')

#对数变换增强的幅度谱
plt.subplot(1,5,5);plt.imshow(imgdft_mag_log,vmin=0,vmax=255)
plt.title('Magnitude spectrum,log')
plt.axis('off')

plt.show()

### 示例：采用OpenCV中的DFT函数计算灰度图像的傅里叶频谱
- dst = cv.dft(src[, dst[, flags[, nonzeroRows]]]); Performs a forward or inverse Discrete Fourier transform of a 1D or 2D floating-point array.

- dst = cv.idft(src[, dst[, flags[, nonzeroRows]]]); Calculates the inverse Discrete Fourier Transform of a 1D or 2D array.
- magnitude = cv.magnitude(x, y[, magnitude]); Calculates the magnitude of 2D vectors
- magnitude, angle = cv.cartToPolar(x, y[, magnitude[, angle[, angleInDegrees]]]);同时返回幅度和相位。
- retval=cv.getOptimalDFTSize(vecsize); Returns the optimal DFT size for a given vector size.用于获取优化后的图像尺寸，DFT 的性能优化
- dst = cv.copyMakeBorder(src, top, bottom, left, right, borderType[, dst[, value]]); 扩展图像Forms a border around an image.
- 当数组的大小为某些值时 DFT 的性能会更好。当数组的大小是 2 的指数时 DFT 效率最高。当数组的大小是 2， 3， 5 的倍数时效率也会很高。所以如果你想提高代码的运行效率时，你可以修改输入图像的大小（补 0）。
- 通常cv.dft() 和 cv.idft() 要比 NumPy,SciPy计算速度快。
- 注意：OpenCV的cv.dft()返回频谱的数据格式和数组维数与SciPy的fft.fft()不同。

In [ ]:
#采用OpenCV中的dft函数计算灰度图像的傅里叶频谱

#读入一幅灰度图像
img = cv.imread('.\imagedata\cameraman.tif',0)

#获取图像高/宽大小
M, N = img.shape

#修改输入图像大小，优化DFT计算速度
P = cv.getOptimalDFTSize(M)
Q = cv.getOptimalDFTSize(N)

#在原图像的下bottom、右right，按指定方式扩展 P-M行、Q-N列
imgn = cv.copyMakeBorder(img,top=0,bottom=P-M, \
                             left=0,right=Q-N,borderType=cv.BORDER_REPLICATE)

#计算扩展图像的DFT
imgdft = cv.dft(np.float32(imgn),flags=cv.DFT_COMPLEX_OUTPUT)
#计算图像的幅度谱
imgdft_mag = cv.magnitude(imgdft[:,:,0],imgdft[:,:,1])

#频谱中心化
imgdft_cent = fftshift(imgdft)
#计算图像的中心化幅度谱
imgdft_mag_cent = cv.magnitude(imgdft_cent[:,:,0],imgdft_cent[:,:,1])

#为便于观察图像的幅度谱,采用对数校正增强幅度谱图像
c1 = 255/np.log10(1+ np.max(imgdft_mag));
imgdft_mag_log = c1 * np.log10(1+ imgdft_mag);
c2 = 255/np.log10(1+ np.max(imgdft_mag_cent));
imgdft_mag_cent_log = c2 * np.log10(1+ imgdft_mag_cent);

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#灰度图像
plt.subplot(1,4,1);plt.imshow(img,vmin=0,vmax=255)
plt.title('Input image')
plt.axis('off')

#未增强的中心化幅度谱
plt.subplot(1,4,2);plt.imshow(imgdft_mag_cent)
plt.title('Centered magnitude spectrum')
plt.axis('off')

#对数变换增强的中心化幅度谱
plt.subplot(1,4,3);plt.imshow(imgdft_mag_cent_log,vmin=0,vmax=255)
plt.title('Centered magnitude spectrum,log')
plt.axis('off')
#对数变换增强的幅度谱
plt.subplot(1,4,4);plt.imshow(imgdft_mag_log,vmin=0,vmax=255)
plt.title('Magnitude spectrum,log')
plt.axis('off')

plt.show()

## 4.2 图像频域滤波基础

### 示例：空域5×5均值滤器的频域实现


In [ ]:
#SciPy-空域均值滤器的频域实现

#读入一幅灰度图像
img = io.imread('.\imagedata\cameraman.tif')

#获取图像高/宽大小
M,N = img.shape

#确定并优化图像扩展参数
P = 2*M
Q = 2*N

#构造一个空域5×5均值滤波器
kav5 = np.ones((5, 5), np.float32)/25
#计算均值滤波器的DFT,输出数组大小为P*Q，默认填充0扩展边界
Hp = fft2(kav5,s=(P,Q))
#对滤波器频谱作中心化处理
Hp = fftshift(Hp)

#计算图像的DFT,扩展区域填充0
Fp1 = fft2(img,s=(P,Q))
#对图像频谱作中心化处理
Fp1 = fftshift(Fp1)

#计算扩展图像DFT与均值滤波器幅度谱的乘积
Gp1 = Fp1 * Hp
#去中心化
Gp1 = ifftshift(Gp1)
#计算滤波结果的DFT反变换，并取实部
imgp1 = np.real(ifft2(Gp1))
#把输出图像的数据格式转换为uint8
imgp1 = np.uint8(np.clip(imgp1,0,255))
#截取imgp1左上角与原图像大小相等的区域作为输出
imgout1 = imgp1[0:M,0:N]


#采用'reflect'方式扩展图像(下面扩展M行，右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp2 = fftshift(fft2(imgex))

#计算'reflect'方式扩展图像的DFT与均值滤波器DFT的乘积
Gp2 = Fp2 * Hp
#去中心化
Gp2 = ifftshift(Gp2)
#DFT反变换,并取实部
imgp2 = np.real(ifft2(Gp2))
#把输出图像的数据格式转换为uint8
imgp2 = np.uint8(np.clip(imgp2,0,255))
#截取imgp2左上角与原图像大小相等的区域作为输出
imgout2 = imgp2[0:M,0:N]


#显示结果
plt.figure(figsize=(15,9))
plt.gray()

#原图像
plt.subplot(2,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#0填充扩展图像滤波结果
plt.subplot(2,3,2); plt.imshow(imgout1,vmin=0,vmax=255)
plt.title('Output Image, padding 0') 
plt.axis('off')
#0填充扩展图像滤波结果(未裁剪)
plt.subplot(2,3,3); plt.imshow(imgp1,vmin=0,vmax=255)
plt.title('Output Image, padding 0,not clipped') 
plt.axis('off')

#5×5均值滤波器的中心化幅度谱
plt.subplot(2,3,4); plt.imshow(np.abs(Hp))
plt.title('5×5 average filter spectrum')
plt.axis('off')

#'reflect'方式扩展图像滤波结果
plt.subplot(2,3,5); plt.imshow(imgout2,vmin=0,vmax=255)
plt.title('Output Image, padding reflect') 
plt.axis('off')
#'reflect'方式扩展图像滤波结果(未裁剪)
plt.subplot(2,3,6); plt.imshow(imgp2,vmin=0,vmax=255)
plt.title('Output Image, padding reflect,not clipped') 
plt.axis('off')

plt.show()

In [ ]:
#OpenCV-空域均值滤器的频域实现

#读入一幅灰度图像
img = cv.imread('.\imagedata\cameraman.tif',0)
#获取图像高/宽大小
M, N = img.shape

#构造一个空域5×5均值滤波器
kav5 = np.ones((5, 5), np.float32)/25
#获取滤波器系数数组高/宽大小
S, T = kav5.shape

#确定并优化图像扩展参数
P = cv.getOptimalDFTSize(M+S-1)
Q = cv.getOptimalDFTSize(N+T-1)

#在原图像的下bottom、右right，按指定方式扩展 P-M行、Q-N列,扩展区域填充0
imgp0 = cv.copyMakeBorder(img,top=0,bottom=P-M,left=0,right=Q-N,borderType=cv.BORDER_CONSTANT,value=0)
#计算扩展图像的DFT
Fp1 = cv.dft(np.float32(imgp0),flags=cv.DFT_COMPLEX_OUTPUT)

#计算均值滤波器的DFT
#扩展均值滤波器大小
kav5p = cv.copyMakeBorder(kav5,top=0,bottom=P-S,left=0,right=Q-T,borderType=cv.BORDER_CONSTANT,value=0)
#计算均值滤波器的DFT
Hpt = cv.dft(kav5p,flags=cv.DFT_COMPLEX_OUTPUT)
#计算均值滤波器幅度谱，得到零相移频域滤波器
Hpm = cv.magnitude(Hpt[:,:,0],Hpt[:,:,1])
#堆叠构建P*Q*2数组
Hp = np.dstack((Hpm,Hpm))

#计算扩展图像DFT与均值滤波器幅度谱的乘积
Gp1 =  cv.multiply(Fp1,Hp)

#计算滤波结果的DFT反变换,并取幅值
imgtemp = cv.idft(Gp1,flags=cv.DFT_SCALE)
imgtemp1 = cv.magnitude(imgtemp[:,:,0],imgtemp[:,:,1])
#把输出图像的数据格式转换为uint8
imgtemp1 = np.uint8(np.clip(imgtemp1,0,255))
#截取imgtemp1左上角与原图像大小相等的区域作为输出
imgout1 = imgtemp1[0:M, 0:N]


#在图像的下bottom、右right，按边界复制方式扩展 P-M行、Q-N列
imgpc = cv.copyMakeBorder(img,top=0,bottom=P-M,left=0,right=Q-N,borderType=cv.BORDER_REPLICATE)
#计算扩展图像的DFT
Fp2 = cv.dft(np.float32(imgpc),flags=cv.DFT_COMPLEX_OUTPUT)

#计算扩展图像DFT与均值滤波器幅度谱的乘积
Gp2 =  cv.multiply(Fp2, Hp)

#DFT反变换,并取实部
imgtemp = cv.idft(Gp2, flags=cv.DFT_SCALE)
imgtemp2 = cv.magnitude(imgtemp[:,:,0],imgtemp[:,:,1])
#把输出图像的数据格式转换为uint8
imgtemp2 = np.uint8(np.clip(imgtemp2,0,255))
#截取imgtemp2左上角与原图像大小相等的区域作为输出
imgout2 = imgtemp2[0:M, 0:N]

#显示结果
plt.figure(figsize=(15,9))
plt.gray()

#原图像
plt.subplot(2,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#0填充扩展图像滤波结果
plt.subplot(2,3,2); plt.imshow(imgout1,vmin=0,vmax=255)
plt.title('Output Image, padding 0') 
plt.axis('off')
#0填充扩展图像滤波结果(未裁剪)
plt.subplot(2,3,3); plt.imshow(imgtemp1,vmin=0,vmax=255)
plt.title('Output Image, padding 0,not clipped') 
plt.axis('off')

#5×5均值滤波器的中心化幅度谱
plt.subplot(2,3,4); plt.imshow(fftshift(Hpm))
plt.title('5×5 average filter spectrum')
plt.axis('off')


#BORDER_REPLICATE方式扩展图像滤波结果
plt.subplot(2,3,5); plt.imshow(imgout2,vmin=0,vmax=255)
plt.title('Output Image, BORDER_REPLICATE') 
plt.axis('off')
#BORDER_REPLICATE方式扩展图像滤波结果(未裁剪)
plt.subplot(2,3,6); plt.imshow(imgtemp2,vmin=0,vmax=255)
plt.title('Output Image, BORDER_REPLICATE,not clipped') 
plt.axis('off')

plt.show()

# 4.3 频域图像平滑

## 4.3.1 巴特沃斯低通滤波器
- 由卷积定理可知，频域滤波就是用滤波器函数H(u,v)的系数去“修改”F(u,v)各频率复正弦的幅度值，从而改变图像f(x,y)频率组分的强弱，进而改变其空域形态。
- 就低通滤波器而言，H(u,v)对应低频区域的通带系数接近1，对应高频区域的阻带系数远小于1并趋近0。
- 巴特沃斯低通滤波器的阶次n用于控制过渡带的陡峭程度，n越大，则过渡带越窄、越陡峭，并趋向理想低通滤波器，振铃现象也就越严重。n=1时的一阶巴特沃斯低通滤波器无振铃现象，n=2时的二阶巴特沃斯低通滤波器有轻微的振铃现象，并在有效的低通滤波和可接受的振铃现象之间获得较好的折中平衡。

### 说明：计算频率点(u,v)到频域平面中心零频率点的距离D(u,v)
- 把逐点计算转换为数组计算

In [ ]:
#构建频域平面坐标网格数组，坐标轴定义v列向/u行向
M = 7; N = 5
v = np.arange(-N, N) 
u = np.arange(-M, M)
#构建频域平面坐标网格数组
Va, Ua = np.meshgrid(v, u)
#计算频率点(u,v)到频域平面中心零频率点的距离D(u,v)的平方
Da2 = Ua**2 + Va**2

#显示结果
print('v=',v)
print('u=',u)
print('Va=\n',Va)
print('Ua=\n',Ua)
print('Da2=\n',Da2)

In [ ]:
#采用巴特沃斯低通滤波器的图像频域平滑

#读入一幅灰度图像
img = io.imread('./imagedata/test.jpeg')
cv.cvtColor(img, cv.COLOR_BGR2GRAY)
img = img[:,:,0]

#获取图像高/宽大小
M, N = img.shape

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp = fftshift(fft2(imgex))

#初始化巴特沃斯低通滤波器的阶次
n = 2
#初始化巴特沃斯低通滤波器的截止频率
D0 = 30

#当n=20,D0=30，出现振铃现象，试试看！！！


#计算巴特沃斯低通滤波器频域传递函数
#构建频域平面坐标网格数组，坐标轴定义v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HBlpf = 1/(1+(Da2/D0**2)**n)

#Da = np.sqrt(Ua**2 + Va**2) 
#HBlpf = 1/(1+(Da/D0)**(2*n)) 

#计算图像DFT与巴特沃斯低通滤波器频域传递函数的乘积
Gp = Fp * HBlpf
#去中心化
Gp = ifftshift(Gp)
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp))
#截取imgp左上角与原图像大小相等的区域作为输出
imgout = imgp[0:M,0:N]
#把输出图像的数据类型转换为uint8
imgout = np.uint8(np.clip(imgout,0,255))

#对比频域滤波前后的图像幅度谱
#为便于观察图像的幅度谱,采用对数校正增强幅度谱图像
Fp_mag = exposure.adjust_log(np.abs(Fp), 1)
Gp_mag = exposure.adjust_log(np.abs(Fp * HBlpf))

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#原图像
plt.subplot(2,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#巴特沃斯低通滤波结果
plt.subplot(2,3,2); plt.imshow(imgout,vmin=0,vmax=255)
plt.title('Output Image') 
plt.axis('off')
#巴特沃斯低通滤波器幅度谱
plt.subplot(2,3,3); plt.imshow(np.abs(HBlpf))
plt.title('Butterworth Low Pass Filter Spectrum')
plt.axis('off')

#原图像幅度谱
plt.subplot(2,3,4); plt.imshow(Fp_mag)
plt.title('Input Image Spectrum')
plt.axis('off')
#输出图像幅度谱
plt.subplot(2,3,5); plt.imshow(Gp_mag)
plt.title('Output Image Spectrum')
plt.axis('off')

plt.show()

In [9]:
#cv.imwrite('Cameraman_InputImageSpectrum.jpg',Fp_mag,[cv.IMWRITE_JPEG_QUALITY,100])
#cv.imwrite('Cameraman_OutputImageSpectrum.jpg',Gp_mag,[cv.IMWRITE_JPEG_QUALITY,100])
#cv.imwrite('ButterworthLowPassFilterSpectrum.jpg',np.abs(HBlpf)*255,[cv.IMWRITE_JPEG_QUALITY,100])

## 4.3.2 高斯低通滤波器
- 在截至频率D0相同的情况下，高斯低通滤波器的过渡带比巴特沃斯低通滤波器更平缓，对低频通带和高频阻带的控制不如二阶巴特沃斯低通滤波器那样“紧凑”，导致平滑效果差些。
- 但高斯低通滤波器不会发生振铃现象。

In [ ]:
#采用高斯低通滤波器的图像频域平滑

#读入一幅灰度图像
img = io.imread('.\imagedata\cameraman.tif')
#获取图像高/宽大小
M,N = img.shape

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp = fftshift(fft2(imgex))

#初始化高斯低通滤波器的截止频率
D0 = 30

#计算高斯低通滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HGlpf = np.exp(-Da2/(2*D0**2))

#计算图像DFT与高斯低通滤波器频域传递函数的乘积
Gp = Fp * HGlpf
#去中心化
Gp = ifftshift(Gp)
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp))
#截取imgp左上角与原图像大小相等的区域作为输出
imgout = imgp[0:M,0:N]
#把输出图像的数据类型转换为uint8
imgout = np.uint8(np.clip(imgout,0,255))

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#高斯低通滤波结果
plt.subplot(1,3,2); plt.imshow(imgout,vmin=0,vmax=255)
plt.title('Output Image') 
plt.axis('off')
#高斯低通滤波器幅度谱
plt.subplot(1,3,3); plt.imshow(np.abs(HGlpf))
plt.title('Gaussian Low Pass Filter Spectrum')
plt.axis('off')

plt.show()

# 4.4 频域图像锐化
- 图像锐化（Image Sharpening）通过补偿图像的轮廓细节，增强图像的边缘及灰度跳变部分，使图像变得更清晰。图像锐化突出了图像中物体的边缘、轮廓，提高了物体边缘与周围像素之间的反差，因此也被称为边缘增强。
- 在第3章中给出了图像锐化的空域滤波实现方法，本节介绍图像锐化的频域实现方法。
## 4.4.1 巴特沃斯高通滤波器


In [ ]:
#采用巴特沃斯高通滤波器的图像频域滤波

#读入一幅灰度图像
img = io.imread('.\imagedata\cameraman.tif')
#获取图像高/宽大小
M,N = img.shape

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp = fftshift(fft2(imgex))

#初始化巴特沃斯高通滤波器的阶次
n=2
#初始化巴特沃斯高通滤波器的截止频率
D0 = 30

#计算巴特沃斯低通滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HBhpf = 1- 1/(1+(Da2/D0**2)**n)

#计算图像DFT与巴特沃斯高通滤波器频域传递函数的乘积
Gp = Fp * HBhpf
#去中心化
Gp = ifftshift(Gp)
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp))
#截取imgp左上角与原图像大小相等的区域作为输出
imgout = imgp[0:M,0:N]

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#巴特沃斯高通滤波结果
plt.subplot(1,3,2); plt.imshow(imgout)
plt.title('Output Image') 
plt.axis('off')
#巴特沃斯高通滤波器幅度谱
plt.subplot(1,3,3); plt.imshow(np.abs(HBhpf))
plt.title('Butterworth High Pass Filter Spectrum')
plt.axis('off')

plt.show()

## 4.4.2 高斯高通滤波器

In [ ]:
#采用高斯高通滤波器的图像频域滤波

#读入一幅灰度图像
img = io.imread('.\imagedata\cameraman.tif')
#获取图像高/宽大小
M,N = img.shape

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp = fftshift(fft2(imgex))

#初始化高斯高通滤波器的截止频率
D0 = 30

#计算高斯高通滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HGhpf = 1- np.exp(-Da2/(2*D0**2))

#计算图像DFT与高斯高通滤波器频域传递函数的乘积
Gp = Fp * HGhpf
#去中心化
Gp = ifftshift(Gp)
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp))
#截取imgp左上角与原图像大小相等的区域作为输出
imgout = imgp[0:M,0:N]

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#高斯高通滤波结果
plt.subplot(1,3,2); plt.imshow(imgout)
plt.title('Output Image') 
plt.axis('off')
#高斯高通滤波器幅度谱
plt.subplot(1,3,3); plt.imshow(np.abs(HGhpf))
plt.title('Gaussian High Pass Filter Spectrum')
plt.axis('off')

plt.show()

## 4.4.3 频域拉普拉斯算子的图像锐化

In [ ]:
#采用频域拉普拉斯算子的图像锐化

#读入一幅灰度图像
img = io.imread('./imagedata/moon.tif')
#将图像数据类型转换为浮点型[0,1]
img = util.img_as_float(img)
#获取图像高/宽大小
M,N = img.shape

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp = fftshift(fft2(imgex))

#初始化叠加强度控制因子
alpha = 1;

#计算频域拉普拉斯算子的传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HLap =-4*np.pi*np.pi*Da2

#计算图像DFT与拉普拉斯算子传递函数的乘积
Gp = Fp * HLap
#去中心化
Gp = ifftshift(Gp)
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp))
#截取imgp左上角与原图像大小相等的区域作为输出
imgLap = imgp[0:M,0:N]
#将计算得到拉普拉斯图像imgLap的值映射到[-1,1]之间
imgLap = imgLap/(np.max(np.abs(imgLap)))

#把拉普拉斯图像fL叠加到原图像上
imgout = img - alpha*imgLap

#将锐化结果imgout中小于0的值置为0,大于1的值置为1
#数据格式转换为uint8
imgout = util.img_as_ubyte(np.clip(imgout,0,1))

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#原图像
plt.subplot(1,3,1); plt.imshow(img,vmin=0,vmax=1)
plt.title('Input Image') 
plt.axis('off')
#拉普拉斯锐化结果
plt.subplot(1,3,2); plt.imshow(imgout,vmin=0,vmax=255)
plt.title('Output Image,Laplacian sharpened') 
plt.axis('off')
#拉普拉斯边缘图像
plt.subplot(1,3,3); plt.imshow(imgLap)
plt.title('Laplacian edge image')
plt.axis('off')

plt.show()

## 4.4.4 钝化掩膜图像锐化的频域实现

In [ ]:
#采用高斯高通滤波器的USM频域锐化

#读入一幅灰度图像
img = io.imread('./imagedata/moon.tif')

#获取图像高/宽大小
M,N = img.shape

#叠加强度控制因子
alpha = 2;
#初始化高斯低通滤波器的截止频率
D0 = 80

#采用'reflect'方式扩展图像(下面扩展M行,右面扩张N列)
imgex = np.pad(img,((0,M),(0,N)),mode='reflect')
#计算扩展图像的DFT,并中心化
Fp = fftshift(fft2(imgex))

#计算高斯低通滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
Da2 = Ua**2 + Va**2
HGlpf = np.exp(-Da2/(2*D0**2))

#计算USM锐化图像的频谱
Gp = Fp*(1+ alpha*(1- HGlpf))
#去中心化
Gp = ifftshift(Gp)
#DFT反变换,并取实部
imgp = np.real(ifft2(Gp))
#截取imgp左上角与原图像大小相等的区域作为输出
imgout = imgp[0:M,0:N]
#把输出图像的数据格式转换为uint8
imgout = np.uint8(np.clip(imgout,0,255))

#显示结果
plt.figure(figsize=(12,8))
plt.gray()

#原图像
plt.subplot(1,2,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#USM锐化结果
plt.subplot(1,2,2); plt.imshow(imgout,vmin=0,vmax=255)
plt.title('Output Image,USM sharpened') 
plt.axis('off')

plt.show()

## 4.4.5 同态滤波器

### 示例：同态滤波器(Homomorphic Filter)增强不均匀光照图像

In [ ]:
#同态滤波器(Homomorphic Filter)增强不均匀光照图像

#采用巴特沃斯滤波器(或高斯滤波器)
#读入一幅灰度图像
img = io.imread('./imagedata/tunnel.jpg')
#获取图像高/宽大小
M,N = img.shape

#转换图像数据类型为浮点型
img = np.float32(img)
#图像img取对数,得到imgz
imgz = np.log(img + 1)

#计算对数图像imgz的DFT并中心化
Zp =  fftshift(fft2(imgz,s=(2*M,2*N)))

#初始化同态滤波器参数
rL=0.5    
rH=1.0
#巴特沃斯滤波器的阶次
n = 2    
#滤波器的截止频率
D0 = 0.6 * np.max((2*M,2*N)) 

#计算同态滤波器频域传递函数
#构建频域平面坐标网格数组，v列向/u行向
v = np.arange(-N, N) 
u = np.arange(-M, M)
Va, Ua = np.meshgrid(v, u)
#采用巴特沃斯低通滤波器
Da2 = Ua**2 + Va**2
HBlpf = 1/(1+(Da2/D0**2)**n)
Hof =  rL * HBlpf + rH * (1 - HBlpf)

#采用高斯低通滤波器
# Da2 = Ua**2 + Va**2
# HGlpf = np.exp(-Da2/(2*D0**2))
# Hof =  rL * HGlpf + rH * (1 - HGlpf)

#计算同态滤波后的图像频谱,并去中心化
Sp = ifftshift(Zp * Hof)
#DFT反变换,并取实部
imgp = np.real(ifft2(Sp))
#截取imgp左上角M*N的区域
imgp1 = imgp[0:M,0:N]
#取imgp1自然指数,并减1补偿
imgout = np.exp(imgp1)-1
#把输出图像的数据类型转换为uint8
imgout = util.img_as_ubyte(imgout/np.max(imgout))

#显示结果
plt.figure(figsize=(16,8))
plt.gray()

#原图像
plt.subplot(1,2,1); plt.imshow(img,vmin=0,vmax=255)
plt.title('Input Image') 
plt.axis('off')
#同态滤波器增强结果
plt.subplot(1,2,2); plt.imshow(imgout,vmin=0,vmax=255)
plt.title('Output Image,Homomorphic Filter enhancement') 
plt.axis('off')

plt.show()
#----------------------------------

# the end